# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

15850647

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[15]
print(key_)

'_i


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

21013

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [13]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 10min 19s
Wall time: 18min 56s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [15]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [2]:
model = Word2Vec.load("../../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [3]:
model.wv["feminisme"]

array([ 0.4882631 , -0.7630363 ,  0.89960045,  0.08274518, -0.66952664,
       -0.63944954,  0.29681465, -0.38940808, -0.84433395, -0.61331505,
        0.03818641,  0.27067986,  0.09037364, -0.5052489 , -0.37554574,
        0.46115187, -0.28848308,  0.19568916, -0.14600404, -1.0004997 ,
       -0.7696276 , -1.0028518 ,  0.51365936,  0.01753154,  0.02030152,
       -0.02065339, -0.5656881 , -0.7481401 ,  0.5894241 , -0.5554212 ,
       -0.24436755,  0.07639734], dtype=float32)

In [4]:
model.wv["encyclique"]

array([-0.21568668, -0.34967107,  0.6211056 ,  0.45612392, -0.3591164 ,
       -1.2885113 ,  0.84439534, -0.4721224 , -0.3534061 , -0.99410343,
        0.32898682,  0.88758856, -0.03139785,  1.6499482 ,  0.10959925,
        0.07349145, -0.5578358 ,  0.19416752,  0.8959092 , -0.70751333,
       -0.7367366 , -0.10414939,  0.44380867, -0.13800868, -0.43657535,
        0.1450027 ,  1.00897   , -0.57291836,  0.1822411 , -0.3912063 ,
       -0.45939314, -0.43647513], dtype=float32)

In [5]:
model.wv["flamand"]

array([-3.2264197 ,  1.4261959 ,  2.4447875 ,  1.7163004 ,  1.1475437 ,
       -2.3481548 , -2.2352018 ,  1.6626484 , -1.867952  ,  0.20393479,
       -1.1979015 ,  0.07912292,  0.58430123, -1.6806992 ,  0.2917771 ,
        4.7495317 ,  4.0405726 ,  0.6404994 ,  0.17432456, -2.1204028 ,
       -0.29003972, -3.2188845 ,  0.7438953 ,  2.8352478 , -3.1947725 ,
       -1.0043234 ,  1.7247869 , -1.2723902 , -0.37213582, -6.635543  ,
       -5.967432  , -1.6906818 ], dtype=float32)

### Calculer la similarité entre deux termes

In [6]:
model.wv.similarity("feminisme", "avortement")

0.5897404

In [8]:
model.wv.similarity("encyclique", "pape")

0.39545557

In [11]:
model.wv.similarity("flamand", "mouvement")

0.12313724

### Chercher les mots les plus proches d'un terme donné

In [12]:
model.wv.most_similar("avortement", topn=10)

[('absenco', 0.8906420469284058),
 ('immoralite', 0.8896996974945068),
 ('inexperience', 0.8839101195335388),
 ('unite_allemande', 0.8821031451225281),
 ('intolerance', 0.8820672631263733),
 ('univers_entier', 0.8783442974090576),
 ('inconsequence', 0.8779166340827942),
 ('analogie', 0.8763653635978699),
 ('anticlericalisme', 0.8746008276939392),
 ('hermetisme', 0.8708783984184265)]

In [14]:
model.wv.most_similar("encyclique", topn=10)

[('allocution', 0.8534609079360962),
 ('biographie', 0.8437253832817078),
 ('lecon_inaugurale', 0.8368021845817566),
 ('entrevue', 0.8338295817375183),
 ('interview', 0.826384425163269),
 ('apologie', 0.8244073390960693),
 ('information_parvenue', 0.8235117793083191),
 ('information', 0.8218907713890076),
 ('enigme', 0.8168473243713379),
 ('note_officieuse', 0.8146634697914124)]

In [7]:
model.wv.most_similar("flamand", topn=10)

[('neerlandais', 0.8925080299377441),
 ('traducteur', 0.8318648338317871),
 ('connaissant_francais', 0.8204221129417419),
 ('francais', 0.7979105114936829),
 ('licencie', 0.7831069827079773),
 ('latin', 0.7805007696151733),
 ('gradue', 0.7765233516693115),
 ('masculin', 0.7750252485275269),
 ('allemand', 0.7622877359390259),
 ('sachant_bien_calculer', 0.7613973021507263)]

### Faire des recherches complexes à travers l'espace vectoriel

In [20]:
print(model.wv.most_similar(positive=['feminisme','belgique'], negative=['france']))

[('referendum', 0.89957594871521), ('reglement', 0.8891162872314453), ('depouillement', 0.8826518058776855), ('bilinguisme', 0.8788301944732666), ('plebiscite', 0.8786926865577698), ('nouveau_statut', 0.878625750541687), ('rejet', 0.8723720908164978), ('protocole', 0.8674011826515198), ('gouvernement_autrichien', 0.8672725558280945), ('probleme_allemand', 0.8656004071235657)]


In [15]:
print(model.wv.most_similar(positive=['encyclique','pape', "naissance", 'vie'], negative=['feminisme']))

[('recommandation', 0.8932327032089233), ('chaire', 0.8908542394638062), ('demarche', 0.8899934887886047), ('capitulation', 0.8796667456626892), ('consecration', 0.8786625862121582), ('dynastie', 0.8781051635742188), ('paternite', 0.877077043056488), ('separation', 0.8719485998153687), ('reconciliation', 0.8708667159080505), ('confirmation', 0.86920565366745)]


In [34]:
print(model.wv.most_similar(positive=['flamand','mouvement', 'politique'], negative=['hollande']))

[('caractere', 0.890923023223877), ('ideologique', 0.8199819326400757), ('diagnostic', 0.8167165517807007), ('vocabulaire', 0.8124483227729797), ('prejuge', 0.8116381168365479), ('constitutionnel', 0.8096004724502563), ('fondement', 0.8040412068367004), ('dynamisme', 0.7997615933418274), ('langage', 0.7986372113227844), ('sujet', 0.7966163158416748)]
